#   LLM-Toolkette zum Dokumentenabgleich

In [1]:
# imports
import json
import pandas as pd

import lxml as etree

from toolchain.llm import LLM
from toolchain.util import Utils
from toolchain.file_operations import FileOperations, X83_Parser

path_to_xml:str = 'data/LV.X83'
path_to_txt:str = 'data/Angebot_Lieferant.txt'

## Table of Contents

1. Setup
2. Modell
   1. Parameter
3. Einlesen von XML-Dateien
   1. Einlesen
   2. Erstellen eines dictonaries
4. Einlesen von TXT-Dateien 
   1. Einlesen
   2. Verarbeitung des Textes
5. LLM
   1. Testen des Modells
   2. Vergleich der XML-Daten und der TXT-Daten
6. Ausgabe der Daten
7. CLI

## 1. Setup



Installation von Dependencies:

In [2]:
!bash -c "pip install -e ."

Obtaining file:///Users/Frederic/Work/QNovi/llm_toolchain
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llmtoolchain (pyproject.toml) ... done
  Created wheel for llmtoolchain: filename=llmtoolchain-1.0-0.editable-py3-none-any.whl size=4188 sha256=d3e5307c73f85e372941f3e05591cbe5b8af966ed4f4615dd8582d640642048b
  Stored in directory: /private/var/folders/7y/ygmwwy3d5sd58bd59ybrzylh0000gn/T/pip-ephem-wheel-cache-c5nggdj4/wheels/f1/21/b0/86bcb14e6bf2aca866422583203be0bd46ec87d3f726c8f525
Successfully built llmtoolchain
  Attempting uninstall: llmtoolchain
    Found existing installation: llmtoolchain 1.0
    Uninstalling llmtoolchain-1.0:
      Successfully uninstalled llmtoolchain-1.0


Modell erstellen

In [3]:
!bash -c "ollama create toolchain_model -f toolchain/modelfiles/model"

gathering model components 
using existing layer sha256:dde5aa3fc5ffc17176b5e8bdc82f587b24b2678c6c66101bf7da77af9f7ccdff 
using existing layer sha256:966de95ca8a62200913e3f8bfbf84c8494536f1b94b49166851e76644e966396 
using existing layer sha256:fcc5a6bec9daf9b561a68827b67ab6088e1dba9d1fa2a50d7bbcc8384e0a265d 
using existing layer sha256:a70ff7e570d97baaf4e62ac6e6ad9975e04caa6d900d3742d37698494479e0cd 
using existing layer sha256:37356fdc1506250840cb1f907e76fd85f0063ea67edc31cf562a72819016c3ee 
using existing layer sha256:d9a23e71e4855bd6a069dc1001ecfb57e78fa6e4bec5fd423f5b8db9b2d48fd8 
writing manifest 
success 


überprüfen

In [4]:
!bash -c "ollama list"

NAME                      ID              SIZE      MODIFIED               
toolchain_model:latest    cb7ba24436a0    2.0 GB    Less than a second ago    
custom_model:latest       ce4fd6811715    2.0 GB    2 days ago                
llama3.2:latest           a80c4f17acd5    2.0 GB    2 days ago                
llama3.1:latest           42182419e950    4.7 GB    5 months ago              


## 2. Modell 

### 2.1 PARAMETER

```python
FROM llama3.2
PARAMETER temperature 0.22
PARAMETER repeat_last_n -1
PARAMETER top_p .1
PARAMETER top_k 25
PARAMETER num_ctx 1024
PARAMETER num_predict 256 
SYSTEM """
    DO NOT HALLUCINATE 
"""
```


## 3. Einlesen von XML-Dateien

### 3.1 Einlesen

In [5]:
root = FileOperations.parse_xml_file(path_to_xml)

In [6]:
xml_parser = X83_Parser(root)
xml_dict:dict = xml_parser.create_dict()

for key, value in xml_dict.items():
    if isinstance(value,dict):
        print(f"{key}")
        for k, v in value.items(): 
            print(f"\t{k}:{v}")
    print("\n\n")

AAAAAAAAIBNBCLNI
	Qty:1.000
	QU:St
	ComplTSB:Yes
	Description: Pufferspeicher  für Innenaufstellung mit Sonderausstattung für alle  Warmwasser-Zentralheizungsanlagen.  Verwendbar für:  festbrennstoffe-, ölgefeuerte Heizkessel, Wärme-  pumpen, Solaranlagen, Gas- oder Elektrodurchlauf-  erhitzer. Gefertigt aus Qualitätsstahl S235JR+AR,   eingestuft nach Druckgeräterichtlinie 2014/68/EU  in Bereich Artikel 4, Absatz 3  (Betriebsmedium: Wasser ohne Gaspolster).  Außen mit Rostschutzanstrich, innen roh.  Technische Daten   Alle Maße in mm ohne Isolierung  Inhalt:   3.133 Liter  Durchmesser:  1.400 mm  Höhe (Oberkante Entlüftung):  2.350 mm  Kippmaß:  2.477 mm  Gewicht (ohne Einbauten):  ca. 425 kg  Betriebsdruck (min.):    3 bar  Betriebsdruck (max.):    6 bar  Prüfdruck:   10 bar  Betriebstemperatur (min.):  20 °C  Betriebstemperatur (max.):  95 °C  Ausrüstung:  12  Muffen nach Wahl 1/2", 1", 1 1/2"  1   Entlüftung bis 1 1/2" 50 mm  4   Flansch DN 80, PN 16  4   Düsenrohr NW 100 / Bogenroh

## 4. Einlesen von TXT-Dateien 

### 4.1 Einlesen

In [7]:
content:str = FileOperations.parse_txt_file(path_to_txt)

### 4.2 Verarbeiten des Textes

In [8]:
# text in Abschnitte unterteilen
sections:list = content.split("\n\n\n")
sections:list = [section.strip("\n") for section in sections]
sections

['1.1.10\n(000070)\nJuratherm Sonderspeicher 3133 Liter\nPufferspeicher für Innenaufstellung mit\nSonderausstattung für alle Warmwasser-\nZentralheizungsanlagen. Verwendbar für:\nfestbrennstoffe-, ölgefeuerte Heizkessel,\nWärmepumpen, Solaranlagen, Gas- oder\nElektrodurchlauferhitzer. Gefertigt aus Qualitätsstahl\nS235JR+AR eingestuft nach Druckgeräterichtlinie\n2014/68/EU in Bereich Artikel 4, Absatz 3\n(Betriebsmedium: Wasser ohne Gaspolster). Außen mit\nRostschutzanstrich, innen roh.\nTechnische Daten Alle Maße in mm ohne Isolierung\nInhalt: 3133 Liter\nDurchmesser: 1400 mm\nHöhe (Oberkante Entlüftung): 2350 mm\nKippmaß: 2477 mm\nGewicht (ohne Einbauten): ca. 425 kg\nBetriebsdruck (min.): 0 bar\nBetriebsdruck (max.): 6 bar\nPrüfdruck: 10 bar\nBetriebstemperatur (min.): 20 °C\nBetriebstemperatur (max.): 95 °C\nAusrüstung\n12x Muffen nach Wahl 1/2", 1", 1 1/2"\n1x Entlüftung 1 1/2" 50 mm\n4x Flansch DN80 / PN6\n4x Bogenrohr DN100 für Flansch DN80, nach oben\nbzw. unten zum Klöpperbode

## 5. LLM



In [9]:
languageModel:LLM = LLM('toolchain_model')

### 5.1 Testen des Modells

In [10]:
prompt:str = "Are you ready compare some data ?"
response:str = languageModel.get_response(prompt).response
print(response)

I'm ready to compare some data. What kind of data would you like to compare, and what are the two datasets you'd like to compare them against?


### 5.2 Vergleich der XML-Daten und der TXT-Daten


In [11]:
prompts:list = []
for i,value in enumerate(xml_dict.values()):
    prompts.append(Utils.create_prompt(value['Description'], sections[i]))

In [12]:
data:list = [languageModel.get_response(prompt).response for prompt in prompts]

## 6. Ausgabe der Daten

In [13]:
df:pd.DataFrame = Utils.create_dataframe_from_responses(data)
df.head()

,is_correct,llm_feedback
0,True,"Both descriptions describe the same product, a..."
1,False,The descriptions are not entirely congruent du...
2,True,Both descriptions match in terms of product de...


In [14]:
FileOperations.write_to_xls(df)

2025-02-21 02:25:11.713 | INFO     | toolchain.file_operations:write_to_xls:71 - Data successfully written to output.xls, sheet 'Sheet1'.


True

## 7. CLI 

Um die Toolchain vom Terminal aus zu nutzen, kann folgender Befehl verwendet werden. 
Dabei ist wichtig die Reinfolge der Argumente zu beachten, erst xml dann txt. 
Der Pfad zur Ausgabedatei ist optional. 

In [15]:
#!bash -c "python run.py "data/LV.X83" "data/Angebot_Lieferant.txt" "output.xls""